# Propagation comparisons between two versions of hcipy

Changing back and forth between conda environments while debugging.

In [ ]:
import os
from astropy.io import fits
import asdf

import hcipy
from matplotlib.colors import TwoSlopeNorm
import matplotlib.pyplot as plt
import numpy as np

from pastis.config import CONFIG_PASTIS
from pastis.simulators.scda_telescopes import HexRingAPLC
from pastis import util

In [ ]:
zerocen = TwoSlopeNorm(vcenter=0)

In [ ]:
NUM_RINGS = 2

test_data_dir = os.path.join(util.find_package_location(), 'tests')
matrix_path2 = os.path.join(test_data_dir, 'data', 'pastis_matrices', 'Hex2_matrix_piston-only.fits')
HEX2_INTENSITY_MATRIX = fits.getdata(matrix_path2)
NSEG_HEX2 = 3 * NUM_RINGS * (NUM_RINGS + 1) + 1

# Set up 2-Hex simulator instance and its basic properties
optics_input = os.path.join(util.find_repo_location(), 'data', 'SCDA')
sampling = CONFIG_PASTIS.getfloat('HexRingTelescope', 'sampling')
hex2 = HexRingAPLC(optics_input, NUM_RINGS, sampling)

## Aperture comparison

In [ ]:
hcipy.imshow_field(hex2.aperture)

In [ ]:
hcipy.write_fits(hex2.aperture, 'pastis-elt.fits')

In [ ]:
pastis = hcipy.read_fits('pastis.fits')

In [ ]:
this = hex2.aperture.shaped
print(this.shape)
print(pastis.shape)

In [ ]:
plt.imshow(this - pastis)
plt.colorbar()
print(np.max(this - pastis))

## Evaluated segments comparison

In [ ]:
one = hex2._create_evaluated_segment_grid()

In [ ]:
hcipy.imshow_field(one[0])

In [ ]:
for i in range(len(one)):
    hcipy.write_fits(one[i], f'seg{i}_pastis.fits')

In [ ]:
segs_pastis = []
for i in range(len(one)):
    segs_pastis.append(one[i].shaped)

In [ ]:
segs_pastis_elt = []
for i in range(len(one)):
    segs_pastis_elt.append(hcipy.read_fits(f'seg{i}_pastis-elt.fits'))

In [ ]:
index = 3
plt.imshow(segs_pastis[index] - segs_pastis_elt[index])
print(np.sum(segs_pastis[index] - segs_pastis_elt[index]))
print(np.min(segs_pastis[index] - segs_pastis_elt[index]))
print(np.max(segs_pastis[index] - segs_pastis_elt[index]))

## Segmented mirror influence functions comparison

In [ ]:
hex2.create_segmented_mirror(3)

In [ ]:
this = hex2.sm.influence_functions

In [ ]:
type(this)

In [ ]:
hcipy.write_mode_basis(this, 'basis_pastis-elt.asdf')

In [ ]:
pastis_file = asdf.open('basis_pastis.asdf')
pastis_elt_file = asdf.open('basis_pastis-elt.asdf')

In [ ]:
pastis_file.info()

In [ ]:
pastis = pastis_file['mode_basis']
pastis_elt = pastis_elt_file['mode_basis']

In [ ]:
print(pastis['coords'])
print(pastis_elt['coords'])

In [ ]:
print(pastis['zero'])
print(pastis_elt['zero'])

In [ ]:
pastis

In [ ]:
pastis_elt

In [ ]:
mat_pa = pastis['transformation_matrix']
mat_elt = pastis_elt['transformation_matrix']

In [ ]:
print(mat_pa.shape)
print(mat_elt.shape)

In [ ]:
index = 47
pas = mat_pa[:,index].reshape(1024, 1024)
elt = mat_elt[:,index].reshape(1024, 1024)
diff = pas - elt
plt.imshow(diff, cmap='RdBu', origin='lower')
plt.colorbar()
print(np.sum(diff))
print(np.min(diff))
print(np.max(diff))

## Propagated efields comparison

In [ ]:
unaberrated_coro_psf, direct = hex2.calc_psf(ref=True, norm_one_photon=True)
NORM = np.max(direct)
EFIELD_REF, _inter = hex2.calc_psf(return_intermediate='efield', norm_one_photon=True)

In [ ]:
NORM

NORM for pastis_elt is 0.0137349  
NORM for pastis is 0.0137349

In [ ]:
hcipy.write_fits(unaberrated_coro_psf, 'unaberrated_coro_psf_pastis.fits')
hcipy.write_fits(direct, 'direct_pastis.fits')
hcipy.write_fits(EFIELD_REF.real, 'efield-ref-REAL_pastis.fits')
hcipy.write_fits(EFIELD_REF.imag, 'efield-ref-IMAG_pastis.fits')

In [ ]:
_inter

In [ ]:
hcipy.write_fits(_inter['seg_mirror'].real, 'real_seg_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['zernike_mirror'].real, 'real_zernike_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['dm'].real, 'real_dm_pastis-elt.fits')
hcipy.write_fits(_inter['harris_seg_mirror'].real, 'real_harris_seg_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['ripple_mirror'].real, 'real_ripple_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['active_pupil'].real, 'real_active_pupil_pastis-elt.fits')
hcipy.write_fits(_inter['apod'].real, 'real_apod_pastis-elt.fits')
hcipy.write_fits(_inter['before_fpm'].real, 'real_before_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['after_fpm'].real, 'real_after_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['before_lyot'].real, 'real_before_lyot_pastis-elt.fits')
hcipy.write_fits(_inter['after_lyot'].real, 'real_after_lyot_pastis-elt.fits')

In [ ]:
hcipy.write_fits(_inter['seg_mirror'].imag, 'imag_seg_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['zernike_mirror'].imag, 'imag_zernike_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['dm'].imag, 'imag_dm_pastis-elt.fits')
hcipy.write_fits(_inter['harris_seg_mirror'].imag, 'imag_harris_seg_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['ripple_mirror'].imag, 'imag_ripple_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['active_pupil'].imag, 'imag_active_pupil_pastis-elt.fits')
hcipy.write_fits(_inter['apod'].imag, 'imag_apod_pastis-elt.fits')
hcipy.write_fits(_inter['before_fpm'].imag, 'imag_before_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['after_fpm'].imag, 'imag_after_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['before_lyot'].imag, 'imag_before_lyot_pastis-elt.fits')
hcipy.write_fits(_inter['after_lyot'].imag, 'imag_after_lyot_pastis-elt.fits')

Read

In [ ]:
elt_unaber = hcipy.read_fits('unaberrated_coro_psf_pastis-elt.fits')
elt_direct = hcipy.read_fits('direct_pastis-elt.fits')
elt_real = hcipy.read_fits('efield-ref-REAL_pastis-elt.fits')
elt_imag = hcipy.read_fits('efield-ref-IMAG_pastis-elt.fits')

In [ ]:
plt.imshow(unaberrated_coro_psf.shaped - elt_unaber, cmap='RdBu', origin='lower')
plt.title("Unaberrated coro img, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.imshow(direct.shaped - elt_direct, cmap='RdBu', origin='lower')
plt.title("Direct img, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.imshow(EFIELD_REF.real.shaped - elt_real, cmap='RdBu', origin='lower')
plt.title("ref efield real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.imshow(EFIELD_REF.imag.shaped - elt_imag, cmap='RdBu', origin='lower')
plt.title("ref efield imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
seg_mirror_elt_real = hcipy.read_fits('real_seg_mirror_pastis-elt.fits')
zernike_elt_real = hcipy.read_fits('real_zernike_mirror_pastis-elt.fits')
dm_elt_real = hcipy.read_fits('real_dm_pastis-elt.fits')
harris_elt_real = hcipy.read_fits('real_harris_seg_mirror_pastis-elt.fits')
ripple_elt_real = hcipy.read_fits('real_ripple_mirror_pastis-elt.fits')
active_elt_real = hcipy.read_fits('real_active_pupil_pastis-elt.fits')
apod_elt_real = hcipy.read_fits('real_apod_pastis-elt.fits')
before_fpm_elt_real = hcipy.read_fits('real_before_fpm_pastis-elt.fits')
after_fpm_elt_real = hcipy.read_fits('real_after_fpm_pastis-elt.fits')
before_ls_elt_real = hcipy.read_fits('real_before_lyot_pastis-elt.fits')
after_ls_elt_real = hcipy.read_fits('real_after_lyot_pastis-elt.fits')

In [ ]:
seg_mirror_elt_imag = hcipy.read_fits('imag_seg_mirror_pastis-elt.fits')
zernike_elt_imag = hcipy.read_fits('imag_zernike_mirror_pastis-elt.fits')
dm_elt_imag = hcipy.read_fits('imag_dm_pastis-elt.fits')
harris_elt_imag = hcipy.read_fits('imag_harris_seg_mirror_pastis-elt.fits')
ripple_elt_imag = hcipy.read_fits('imag_ripple_mirror_pastis-elt.fits')
active_elt_imag = hcipy.read_fits('imag_active_pupil_pastis-elt.fits')
apod_elt_imag = hcipy.read_fits('imag_apod_pastis-elt.fits')
before_fpm_elt_imag = hcipy.read_fits('imag_before_fpm_pastis-elt.fits')
after_fpm_elt_imag = hcipy.read_fits('imag_after_fpm_pastis-elt.fits')
before_ls_elt_imag = hcipy.read_fits('imag_before_lyot_pastis-elt.fits')
after_ls_elt_imag = hcipy.read_fits('imag_after_lyot_pastis-elt.fits')

In [ ]:
elt_real = [seg_mirror_elt_real, zernike_elt_real, dm_elt_real, harris_elt_real, ripple_elt_real,
            active_elt_real, apod_elt_real, before_fpm_elt_real, after_fpm_elt_real, before_ls_elt_real, after_ls_elt_real]

In [ ]:
elt_imag = [seg_mirror_elt_imag, zernike_elt_imag, dm_elt_imag, harris_elt_imag, ripple_elt_imag,
            active_elt_imag, apod_elt_imag, before_fpm_elt_imag, after_fpm_elt_imag, before_ls_elt_imag, after_ls_elt_imag]

In [ ]:
hcipy.write_fits(_inter['seg_mirror'].imag, 'imag_seg_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['zernike_mirror'].imag, 'imag_zernike_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['dm'].imag, 'imag_dm_pastis-elt.fits')
hcipy.write_fits(_inter['harris_seg_mirror'].imag, 'imag_harris_seg_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['ripple_mirror'].imag, 'imag_ripple_mirror_pastis-elt.fits')
hcipy.write_fits(_inter['active_pupil'].imag, 'imag_active_pupil_pastis-elt.fits')
hcipy.write_fits(_inter['apod'].imag, 'imag_apod_pastis-elt.fits')
hcipy.write_fits(_inter['before_fpm'].imag, 'imag_before_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['after_fpm'].imag, 'imag_after_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['before_lyot'].imag, 'imag_before_lyot_pastis-elt.fits')
hcipy.write_fits(_inter['after_lyot'].imag, 'imag_after_lyot_pastis-elt.fits')

In [ ]:
pas_wfs = [_inter['seg_mirror'], _inter['zernike_mirror'], _inter['dm'], _inter['harris_seg_mirror'],
           _inter['ripple_mirror'], _inter['active_pupil'], _inter['apod'], _inter['before_fpm'],
           _inter['after_fpm'], _inter['before_lyot'], _inter['after_lyot']]

In [ ]:
plt.figure(figsize=(10,20))
for i in range(len(elt_real)):
    size = int(np.sqrt(pas_wfs[i].real.shape)[0])
    
    plt.subplot(11, 2, i*2 + 1)
    plt.imshow(pas_wfs[i].real.reshape(size, size) - elt_real[i], cmap='RdBu', origin='lower')
    plt.colorbar()
    plt.subplot(11, 2, i*2 + 2)
    plt.imshow(pas_wfs[i].imag.reshape(size, size) - elt_imag[i], cmap='RdBu', origin='lower')
    plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.imshow(_inter['before_fpm'].real.shaped - before_fpm_elt_real, cmap='RdBu', origin='lower')
plt.title("before FPM real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(_inter['before_fpm'].imag.shaped - before_fpm_elt_imag, cmap='RdBu', origin='lower')
plt.title("before FPM imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.imshow(_inter['before_lyot'].real.shaped - before_ls_elt_real, cmap='RdBu', origin='lower')
plt.title("Lyot plane real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(_inter['before_lyot'].imag.shaped - before_ls_elt_imag, cmap='RdBu', origin='lower')
plt.title("Loyt plane imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

## MWE with hcipy

In [ ]:
grid = hcipy.make_pupil_grid(1000, diameter=15)

In [ ]:
pup = hcipy.make_luvoir_a_aperture()(grid)

In [ ]:
hcipy.imshow_field(pup, cmap='Greys_r')

In [ ]:
focal = hcipy.make_focal_grid(4, 32)
prop = hcipy.FraunhoferPropagator(grid, focal)

In [ ]:
wf_pup = hcipy.Wavefront(pup, 500e-9)

In [ ]:
wf_fp = prop(wf_pup)

In [ ]:
hcipy.imshow_field(wf_fp.intensity, cmap='inferno')

In [ ]:
wavelength = 500e-9 # meter
pupil_diameter = 15
pupil_grid = hcipy.make_pupil_grid(256, 1.2 * pupil_diameter)
aperture_luvoir = hcipy.evaluate_supersampled(hcipy.make_luvoir_a_aperture(), pupil_grid, 8)
wf_luvoir = hcipy.Wavefront(aperture_luvoir, wavelength)

focal_length = 0.5 # meter
spatial_resolution = focal_length / pupil_diameter * wavelength
focal_grid = hcipy.make_focal_grid(8, 12, spatial_resolution=spatial_resolution)

fraunhofer = hcipy.FraunhoferPropagator(pupil_grid, focal_grid, focal_length=focal_length)
img_luvoir = fraunhofer(wf_luvoir)

In [ ]:
hcipy.write_fits(img_luvoir.intensity, 'int_pastis.fits')
hcipy.write_fits(img_luvoir.phase, 'phase_pastis.fits')
hcipy.write_fits(img_luvoir.real, 'real_pastis.fits')
hcipy.write_fits(img_luvoir.imag, 'imag_pastis.fits')

In [ ]:
pas_int = hcipy.read_fits('int_pastis.fits')
pas_phase = hcipy.read_fits('phase_pastis.fits')
pas_real = hcipy.read_fits('real_pastis.fits')
pas_imag = hcipy.read_fits('imag_pastis.fits')

In [ ]:
plt.subplot(2, 2, 1)
plt.imshow(pas_int - img_luvoir.intensity.shaped, cmap='RdBu')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(pas_phase - img_luvoir.phase.shaped, cmap='RdBu')
plt.colorbar()
plt.subplot(2, 2, 3)
plt.imshow(pas_real - img_luvoir.real.shaped, cmap='RdBu')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(pas_imag - img_luvoir.imag.shaped, cmap='RdBu')
plt.colorbar()

## Repeating the same with an aperture from file

In [ ]:
wavelength = 500e-9 # meter
pupil_diameter = 7.258823341
pupil_grid = hcipy.make_pupil_grid(1024, 1.2 * pupil_diameter)

aper_file = fits.getdata('/Users/ilaginja/repos/PASTIS/data/SCDA/2-Hex/masks/TelAp_LUVex_02-Hex_gy_ovsamp04_N1024.fits')
aperture_luvoir = hcipy.Field(aper_file.ravel(), pupil_grid)

wf_luvoir = hcipy.Wavefront(aperture_luvoir, wavelength)

focal_length = 0.5 # meter
spatial_resolution = focal_length / pupil_diameter * wavelength
focal_grid = hcipy.make_focal_grid(8, 12, spatial_resolution=spatial_resolution)

fraunhofer = hcipy.FraunhoferPropagator(pupil_grid, focal_grid, focal_length=focal_length)
img_luvoir = fraunhofer(wf_luvoir)

In [ ]:
hcipy.write_fits(img_luvoir.intensity, 'int_pastis.fits')
hcipy.write_fits(img_luvoir.phase, 'phase_pastis.fits')
hcipy.write_fits(img_luvoir.real, 'real_pastis.fits')
hcipy.write_fits(img_luvoir.imag, 'imag_pastis.fits')

In [ ]:
pas_int = hcipy.read_fits('int_pastis.fits')
pas_phase = hcipy.read_fits('phase_pastis.fits')
pas_real = hcipy.read_fits('real_pastis.fits')
pas_imag = hcipy.read_fits('imag_pastis.fits')

In [ ]:
plt.subplot(2, 2, 1)
plt.imshow(pas_int - img_luvoir.intensity.shaped, cmap='RdBu')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(pas_phase - img_luvoir.phase.shaped, cmap='RdBu')
plt.colorbar()
plt.subplot(2, 2, 3)
plt.imshow(pas_real - img_luvoir.real.shaped, cmap='RdBu')
plt.colorbar()
plt.subplot(2, 2, 4)
plt.imshow(pas_imag - img_luvoir.imag.shaped, cmap='RdBu')
plt.colorbar()

## Maybe it's the propagation normalization?

In [ ]:
unaberrated_coro_psf, direct = hex2.calc_psf(ref=True, norm_one_photon=False)
NORM = np.max(direct)
EFIELD_REF, _inter = hex2.calc_psf(return_intermediate='efield', norm_one_photon=False)

In [ ]:
NORM

NORM for pastis_elt is 1.61546632e+15  
NORM for pastis is 1.61546632e+15

In [ ]:
hcipy.write_fits(unaberrated_coro_psf, 'unaberrated_coro_psf_pastis-elt.fits')
hcipy.write_fits(direct, 'direct_pastis-elt.fits')
hcipy.write_fits(EFIELD_REF.real, 'efield-ref-REAL_pastis-elt.fits')
hcipy.write_fits(EFIELD_REF.imag, 'efield-ref-IMAG_pastis-elt.fits')

In [ ]:
hcipy.write_fits(_inter['before_fpm'].real, 'real_before_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['before_lyot'].real, 'real_before_lyot_pastis-elt.fits')
hcipy.write_fits(_inter['before_fpm'].imag, 'imag_before_fpm_pastis-elt.fits')
hcipy.write_fits(_inter['before_lyot'].imag, 'imag_before_lyot_pastis-elt.fits')

In [ ]:
elt_unaber = hcipy.read_fits('unaberrated_coro_psf_pastis-elt.fits')
elt_direct = hcipy.read_fits('direct_pastis-elt.fits')
elt_real = hcipy.read_fits('efield-ref-REAL_pastis-elt.fits')
elt_imag = hcipy.read_fits('efield-ref-IMAG_pastis-elt.fits')

In [ ]:
before_fpm_elt_real = hcipy.read_fits('real_before_fpm_pastis-elt.fits')
before_ls_elt_real = hcipy.read_fits('real_before_lyot_pastis-elt.fits')
before_fpm_elt_imag = hcipy.read_fits('imag_before_fpm_pastis-elt.fits')
before_ls_elt_imag = hcipy.read_fits('imag_before_lyot_pastis-elt.fits')

In [ ]:
zerocen = TwoSlopeNorm(vcenter=0, vmin=-650000, vmax=500000)
plt.imshow(unaberrated_coro_psf.shaped - elt_unaber, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("Unaberrated coro img, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
zerocen = TwoSlopeNorm(vcenter=0, vmin=-5, vmax=5)
plt.imshow(direct.shaped - elt_direct, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("Direct img, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.imshow(EFIELD_REF.real.shaped - elt_real, cmap='RdBu', origin='lower')
plt.title("ref efield real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
zerocen = TwoSlopeNorm(vcenter=0, vmin=-5, vmax=19)
plt.imshow(EFIELD_REF.imag.shaped - elt_imag, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("ref efield imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 7))
zerocen = TwoSlopeNorm(vcenter=0, vmin=-1e-8, vmax=1e-8)

plt.subplot(1, 2, 1)
plt.imshow(_inter['before_fpm'].real.shaped - before_fpm_elt_real, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("before FPM real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(_inter['before_fpm'].imag.shaped - before_fpm_elt_imag, cmap='RdBu', origin='lower')
plt.title("before FPM imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.imshow(_inter['before_lyot'].real.shaped - before_ls_elt_real, cmap='RdBu', origin='lower')
plt.title("Lyot plane real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(_inter['before_lyot'].imag.shaped - before_ls_elt_imag, cmap='RdBu', origin='lower')
plt.title("Loyt plane imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()